<a href="https://colab.research.google.com/github/lutakrystal305/RecognitionFace/blob/main/Recognition_VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os

drive.mount("/content/gdrive")
path = "/content/gdrive/My Drive/FaceRecognition"
os.chdir(path)
!ls

Mounted at /content/gdrive


In [ ]:
!pip install face_recognition

In [ ]:
!git clone https://github.com/phamdinhkhanh/FacenetDataset.git ./Dataset

Cloning into './Dataset'...
remote: Enumerating objects: 167, done.
remote: Counting objects: 100% (167/167), done.
remote: Compressing objects: 100% (166/166), done.
remote: Total 167 (delta 6), reused 161 (delta 0), pack-reused 0
Receiving objects: 100% (167/167), 28.37 MiB | 11.14 MiB/s, done.
Resolving deltas: 100% (6/6), done.
Checking out files: 100% (155/155), done.


In [ ]:
#convert ảnh blod
def blod_img(img, out_size=(300,300), scaleFactor=1.0, mean=(104, 177, 123)):
  #img:anh dau vao
  #out_size: kich thuoc anh dau ra
  #scaleFactor: scale anh
  imgBlod = cv2.dnn.blodFromImage(img,
                                  scaleFactor= ScaleFactor,
                                  size = out_size,
                                  mean = mean,
                                  swapRB = False,
                                  crop = False
                                  )
  return imgBlod

In [ ]:
#trich xuat khuon mat
from face_recognition import face_locations
import cv2


IMG_TEST = os.path.join(path, "Dataset/khanh/001.jpg")

def _img_read(image_path):
  img = cv2.imread(image_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

img = _img_read(IMG_TEST)

def extract_bbox(img, single = True):
  #trich xuat cac bounding box chua mat
  #img: anh input (RGB)
  #single: Lay ra 1 face tren 1 anh neu True >< False
  bboxs = face_locations(img)
  if len(bboxs) == 0 :
    return None
  if single:
    bbox = bboxs[0]
    return bbox
  else:
    return bboxs 
    #bbox: (start_y, tart_x, end_y, end_x)

In [ ]:
from google.colab.patches import cv2_imshow

def extract_face(img, bbox, face_scale_thres = (20, 20)):
  #img: anh dau vao RGB
  #bbox: toa do bbox
  #face_scale_thres: ngưỡng kích thước (h, w) của face. Nếu nhỏ hơn ngưỡng này thì loại bỏ face
  h, w = img.shape[:2]
  try:
    (startY, startX, endY, endX) = bbox
  except:
    return None
  minX, maxX = min(startX, endX), max(startX, endX)
  minY, maxY = min(startY, endY), max(startY, endY)
  face = img[minY:maxY, minX:maxX].copy()
  #h, w face
  (fH, fW) = face.shape[:2]

  #dam bao face H, W lon hon ngưỡng
  if fW < face_scale_thres[0] or fH < face_scale_thres[1]:
    return None
  else:
    return face
bbox = extract_bbox(img)
face = extract_face(img, bbox)
cv2_imshow(face)

In [ ]:
from imutils import paths
DATASET_PATH = "./Dataset"

def model_processing(face_scale_thres = (20, 20)):
  img_links = list(paths.list_images(DATASET_PATH))
  #print(img_links)
  img_file = []
  y_labels = []
  faces = []
  total = 0
  for img_link in img_links:
    split_img_link = img_link.split("/")
    #Get label
    name = split_img_link[-2]
    #Read img
    img = img_read(img_link)
    (h, w) = img.shape[:2]
    #Detect vị trí các khuôn mặt trên ảnh (giả sử mỗi ảnh chỉ có duy nhất 1 bouding box)
    bbox = extract_bbox(img)
    if bbox is not None:
      face = extract_face(img, bbox)
    if face is not None:
      faces.append(face)
      y_labels.append(name)
      img_file.append(img_link)
      total += 1
    else:
      next
  print("Total bbox face extracted: {}".format(total))
  return faces, y_labels, img_file
faces, y_labels, img_file = model_processing()

Total bbox face extracted: 150


In [ ]:
import pickle

def save_pickle(obj, path):
  with open(path, 'wb') as f:
    pickle.dump(obj, f)

def load_pickle(path):
  with open(path, 'rb') as f:
    obj = pickle.load(f)
  return obj 

save_pickle(faces, './faces.pkl')
save_pickle(y_labels, './y_labels.pkl')
save_pickle(img_file, "./images_file.pkl")

In [ ]:
%tensorflow_version 2.x

In [ ]:
# train basenetwork triplet loss
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16

def base_network():
  model = VGG16(include_top = True, weights = None)
  dense = Dense(128)(model.layers[-4].output)
  norm2 = Lambda(lambda x: tf.math.l2_normalize(x, axis = 1))(dense)
  model = Model(inputs = [model.input], outputs = [norm2])
  return model

model = base_network()
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [ ]:
faces = load_pickle('./faces.pkl')

In [ ]:
import cv2

faceResizes = []
for face in faces:
  face_rz = cv2.resize(face, (224, 224))
  faceResizes.append(face_rz)
X = np.stack(faceResizes)
X.shape

(150, 224, 224, 3)

In [ ]:
from sklearn.model_selection import train_test_split
ids = np.arange(len(y_labels))

X_train, X_test, y_train, y_test, id_train, id_test = train_test_split(X, y_labels, ids, test_size = 0.2, stratify = y_labels)
print(X_train.shape, X_test.shape)
print(len(y_train), len(y_test))

(120, 224, 224, 3) (30, 224, 224, 3)
120 30


In [ ]:
#pass
id_train = _load_pickle("./id_train.pkl")
id_test = _load_pickle("./id_test.pkl")

X_train, X_test = X[id_train], X[id_test]

print(X_train.shape)
print(X_test.shape)

In [ ]:
!pip install tensorflow-addons

     |████████████████████████████████| 686kB 11.1MB/s 


In [ ]:
import tensorflow_addons as tfa

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tfa.losses.TripletSemiHardLoss())

In [ ]:
gen_train = tf.data.Dataset.from_tensor_slices((X_train, y_train)).repeat().shuffle(1024).batch(32)
gen_train

<BatchDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.uint8, tf.string)>

In [ ]:
history = model.fit(
    gen_train, 
    steps_per_epoch = 50,
    epochs = 10
)

Epoch 1/10
50/50 [==============================] - 85s 786ms/step - loss: 0.9823
Epoch 2/10
50/50 [==============================] - 39s 778ms/step - loss: 0.9724
Epoch 3/10
50/50 [==============================] - 39s 773ms/step - loss: 0.9744
Epoch 4/10
50/50 [==============================] - 39s 773ms/step - loss: 0.8994
Epoch 5/10
50/50 [==============================] - 39s 771ms/step - loss: 0.7993
Epoch 6/10
50/50 [==============================] - 39s 771ms/step - loss: 0.5735
Epoch 7/10
50/50 [==============================] - 39s 770ms/step - loss: 0.3092
Epoch 8/10
50/50 [==============================] - 38s 768ms/step - loss: 0.0883
Epoch 9/10
50/50 [==============================] - 38s 760ms/step - loss: 0.0212
Epoch 10/10
50/50 [==============================] - 38s 753ms/step - loss: 1.5383e-04


In [ ]:
model.save("model/model_triplot.h5")

In [ ]:
X_train_vecs = model.predict(np.expand_dims(X_train, axis = -1))
X_test_vecs = model.predict(np.expand_dims(X_test, axis = -1))

In [ ]:
print(X_train_vecs)
print(X_test_vecs)

[[-0.11954954  0.1537262  -0.08558087 ...  0.09400881 -0.08444931
  -0.07666687]
 [ 0.06019749  0.00480916  0.06774399 ... -0.05751166  0.04855407
  -0.13054699]
 [ 0.14999926 -0.12594655  0.09663776 ... -0.14260794  0.03878479
   0.08426335]
 ...
 [-0.06356598 -0.00905482 -0.07411133 ...  0.03857066 -0.07523076
   0.11108229]
 [ 0.14241725 -0.1276752   0.09787145 ... -0.13880822  0.03496119
   0.09225871]
 [ 0.06076735 -0.00297026  0.07164863 ... -0.04501938  0.07183451
  -0.11815221]]
[[-0.00317157 -0.05532013 -0.00523854 ... -0.04469575 -0.07547715
   0.12759933]
 [-0.07092148  0.00448144 -0.07834631 ...  0.04256755 -0.08728496
   0.10260331]
 [-0.10848095  0.16899341 -0.08719121 ...  0.08135311 -0.11681359
  -0.05819976]
 ...
 [ 0.00651818  0.10738019  0.00290288 ...  0.00656187 -0.02730782
  -0.13188297]
 [-0.01216903 -0.05655323 -0.0328498  ... -0.00169127 -0.04251923
   0.13646832]
 [ 0.04651064  0.02156485  0.0367122  ... -0.01024398  0.09366239
  -0.09285373]]


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def _most_similarity(embed_vecs, vec, labels):
  sim = cosine_similarity(embed_vecs, vec)
  #print(sim)
  sim = np.squeeze(sim, axis = 1)
  #print(sim)
  argmax = np.argsort(sim)[::-1][:1]
  print(np.argsort(sim))
  print(argmax)
  label = [labels[idx] for idx in argmax][0]
  return label

# Lấy ngẫu nhiên một bức ảnh trong test
vec = X_test_vecs[11].reshape(1, -1)
print(vec)
# Tìm kiếm ảnh gần nhất
_most_similarity(X_train_vecs, vec, y_train)

[[-0.10198195  0.01891373 -0.03995753 -0.03126528  0.03406004  0.12153162
  -0.22810574  0.11938945  0.08716784  0.04238087  0.08364395  0.00182744
  -0.08855332  0.03732689 -0.03839699  0.05742324  0.10394975 -0.0994033
   0.06230547 -0.12553385  0.14651035  0.09402804 -0.10167943  0.07665204
  -0.05328413  0.09440935  0.04773529 -0.08056636 -0.08337552 -0.08992729
   0.09883367  0.12966703  0.00346717 -0.00072257  0.12620616 -0.14960973
  -0.01529338  0.01280139 -0.0137792  -0.01871005  0.05198076 -0.04074344
   0.09817506 -0.02225562  0.02818328  0.1187901   0.00270364 -0.12380289
  -0.14257649 -0.13723542 -0.08699587  0.15728574  0.00069981 -0.08951652
  -0.00656995 -0.11785878  0.08520347 -0.14822945  0.05208645 -0.10638969
   0.09602345 -0.118255    0.14629723 -0.06421918  0.04348839  0.00436473
   0.16100512  0.0788284   0.01693978  0.04275547 -0.18311399 -0.10234705
   0.10082828  0.11655971  0.05040074 -0.07443805  0.00669349 -0.13321783
   0.15690416 -0.0889652  -0.16470253 -

'baejun'

In [ ]:
from sklearn.metrics import accuracy_score
y_preds = []
for vec in X_test_vecs:
  vec = vec.reshape(1, -1)
  y_pred = _most_similarity(X_train_vecs, vec, y_train)
  #print(y_pred)
  y_preds.append(y_pred)

print(accuracy_score(y_preds, y_test))

baejun
baejun
sontung
thaotam
sontung
khanh
thaotam
khanh
chipu
chipu
thaotam
baejun
chipu
sontung
sontung
chipu
thaotam
khanh
baejun
baejun
baejun
baejun
khanh
khanh
chipu
chipu
baejun
chipu
baejun
chipu
0.6


In [ ]:
from sklearn.metrics import accuracy_score
y_preds = []
for vec in X_train_vecs:
  vec = vec.reshape(1, -1)
  y_pred = _most_similarity(X_train_vecs, vec, y_train)
  y_preds.append(y_pred)

print(accuracy_score(y_preds, y_train))

1.0


In [ ]:
z = X_test_vecs[11].reshape(-1,1)